# MNIST
Train a convnet classifier on the MNIST dataset and visualize cost function and first layer filters using Agnez app.

### Dependencies:
    Agnez app:
        node.js, coffeescript
    Deep learning:
        Keras

### Usage:
1. Install Agnez app:  
    `git clone --recursive https://github.com/AgnezIO/agnez_app.git`  
    `cd agnez_app`  
    `make`  
2. Run the app  
    `make run`
3. Run this notebook
4. Check results on http://localhost:3000

In [ ]:
%matplotlib inline

'''
This code was modified from: https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py

Train a simple convnet on the MNIST dataset.
Run on GPU: THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python mnist_cnn.py
Get to 99.25% test accuracy after 12 epochs (there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils

from agnez import img_grid
from agnez.app_callbacks import LossPlot, VisualizeConvWeights

# Define model

In [ ]:
batch_size = 128
nb_classes = 10
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

# the data, shuffled and split between tran and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()

model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(1, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adadelta')

# Agnez clients

In [ ]:
# here we are using the default values for `name`, `description` and `app_url`

loss_plot = LossPlot(name='Loss plot', description='train (blue) and validation (green) learning curves',
                     position=0, app_url='http://localhost:3000/api/v1/values')

conv_weights = VisualizeConvWeights(weights=model.layers[0].W, name='Conv weights',
                                    description='weights of convolutional layer', position=1,
                                    app_url='http://localhost:3000/api/v1/values')

# Train

In [ ]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          show_accuracy=True, verbose=1, validation_data=(X_test, Y_test),
          callbacks=[loss_plot, conv_weights])